In [1]:
from pathlib import Path
from dataset import SingleChannelNDIDatasetContrastiveLearningWithAug, get_CNI_tensor
from torch.utils.data import DataLoader
from utils import load_checkpoints
import torchvision
from torch import nn as nn
from models.wd_model import CompositeResNet
from models.core_model import RetrievalModel
import torch

In [2]:
def get_model(key_word, pretrained=False):
    if key_word == 'ResNet50':
        # Set ResNet50 for clarifying the effectiveness of Code
        if pretrained:
            model = torchvision.models.resnet50(weights=torchvision.models.ResNet50_Weights.IMAGENET1K_V2)
        else:
            model = torchvision.models.resnet50(weights=None)
        model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
        model.fc = nn.Linear(model.fc.in_features, 512)
        return model
    # if 'ViT' in key_word:
    #     if pretrained:
    #         model = unicom.load(key_word)[0]
    #     else:
    #         model = build_model(key_word)
        # return model

In [3]:
ORIGINAL_IMAGE = '../datasets/NDI_images/Integreted/Observed/'
EXTRA_IMAGE = '../datasets/NDI_images/Integreted/extra observed/cropped/'
TARGET_IMAGE = '../datasets/NDI_images/Integreted/Calculated/'


original_images = list(sorted(list(map(str, list(Path(ORIGINAL_IMAGE).glob('*.jpg'))))))
target_images = list(sorted(list(map(str, list(Path(TARGET_IMAGE).glob('*.jpg'))))))
images = list(reversed(list(zip(original_images, target_images))))
images = (images, 0)
dataset = SingleChannelNDIDatasetContrastiveLearningWithAug(images, False, 200)
dataloader = DataLoader(dataset, len(dataset), shuffle=False, drop_last=True)

In [4]:
model = get_model('ResNet50', pretrained=True)
model = load_checkpoints(model, './checkpoints/ImageNet_ALL_CHECK_400_Epoch.pth')
model = CompositeResNet(model, 2)
model = RetrievalModel(model)
model.load_state_dict(torch.load('./checkpoints/Best 6th Fold.pth')['state_dict'])
model.cuda()

RetrievalModel(
  (encoder_q): CompositeResNet(
    (model): ResNet(
      (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
      (layer1): Sequential(
        (0): Bottleneck(
          (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          (relu)

In [39]:
device = torch.device('cuda')
target_tensor = get_CNI_tensor(device, 200)
for origin, target, label in dataloader:
    origin, target, label = origin.cuda(), target.cuda(), label.cuda()
    em_ori, em_tar = model(origin, target_tensor)
    em_ori, em_tar = em_ori[0], em_tar[0]
    sim_mat = model.get_similarity_matrix(em_ori, em_tar)

In [40]:
topk = sim_mat.topk(10, 1)[1]
pred_labels = topk.t()
correct = pred_labels.eq(label.view(1, -1).expand_as(pred_labels))

In [41]:
correct.sum(dim=0)

tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0,
        0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], device='cuda:0')

In [42]:
label[correct.sum(dim=0) == 0]

tensor([126, 123, 118, 116, 115, 114, 112, 111], device='cuda:0')

In [40]:
success = []
failure = []
for i in range(len(label)):
    if label[i] in topk[i]:
        success.append(label[i])
    else:
        failure.append(label[i])


In [41]:
len(success)

9

In [29]:
label

tensor([ 98,  97,  96,  95,  94,  93,  92,  91,  90,  89,   8,  88,  87,  86,
         85,  84,  83,  82,  81,  80,  79,   7,  78,  77,  76,  75,  74,  73,
         72,  71,  70,  69,   6,  68,  67,  66,  65,  64,  63,  62,  61,  60,
         59,   5,  58,  57,  56,  55,  54,  53,  52,  51,  50,  49,   4,  48,
         47,  46,  45,  44,  43,  42,  41,  40,  39,   3,  38,  37,  36,  35,
         34,  33,  32,  31,  30,  29,   2,  28,  27,  26,  25,  24,  23,  22,
         21,  20,  19,   1,  18, 183, 182, 181, 180, 179,  17, 178, 177, 176,
        175, 174, 173, 172, 171, 170, 169,  16, 168, 167, 166, 165, 164, 163,
        162, 161, 160, 159,  15, 158, 157, 156, 155, 154, 153, 152, 151, 150,
        149,  14, 148, 147, 146, 145, 144, 143, 142, 141, 140, 139,  13, 138,
        137, 136, 135, 134, 133, 132, 131, 130, 129,  12, 128, 127, 126, 125,
        124, 123, 122, 121, 120, 119,  11, 118, 117, 116, 115, 114, 113, 112,
        111, 110, 109,  10, 108, 107, 106, 105, 104, 103, 102, 1

In [5]:
failures = [92, 80, 77, 74, 72, 68, 65, 61, 57, 55, 49, 47, 41, 39, 3, 38, 34, 161, 28, 26, 25, 24, 21, 20, 19, 183, 182, 173, 170, 157, 154, 149, 143, 140, 136, 126, 123, 118, 116, 115, 114, 112, 111]
failures.sort()

In [11]:
failures

[3,
 19,
 20,
 21,
 24,
 25,
 26,
 28,
 34,
 38,
 39,
 41,
 47,
 49,
 55,
 57,
 61,
 65,
 68,
 72,
 74,
 77,
 80,
 92,
 111,
 112,
 114,
 115,
 116,
 118,
 123,
 126,
 136,
 140,
 143,
 149,
 154,
 157,
 161,
 170,
 173,
 182,
 183]

In [17]:
original_root = '/'.join(original_images[0].split('/')[:-1])
target_root = '/'.join(target_images[0].split('/')[:-1])
failure_observed = []
failure_calculated = []
for example in failures:
    failure_observed.append(original_root + '/' + str(example) + '.jpg')
    failure_calculated.append(target_root + '/' + str(example) + '.jpg')
    # for observed_image in original_images:
    #     if str(example) in observed_image:
    #         failure_observed.append(observed_image)
    #         break
    # for calculated_image in target_images:
    #     if str(example) in calculated_image:
    #         failure_calculated.append(calculated_image)
    #         break


In [20]:
import os
import shutil

tgt_path = "/import/gp-home.cal/duanct/ProjectFolder/datasets/NDI_images/failure matching examples/"

for image in failure_observed:
    name = image.split('/')[-1].split('.')[0]
    target_file = tgt_path + name + '_observed.jpg'
    shutil.copyfile(image, target_file)
for image in failure_calculated:
    name = image.split('/')[-1].split('.')[0]
    target_file = tgt_path + name + '_calculated.jpg'
    shutil.copyfile(image, target_file)